# Ngram Dict insert to Mongo

In [2]:
#-*- utf-8 -*-
import codecs
import operator
import os
import re
import _uniout

def no_court(content):
    try:
        court_no = {u"臺北地方":100,u"士林地方":111,u"新北地方":220,u"宜蘭地方":260,u"基隆地方":200,u"桃園地方":320,u"新竹地方":300,u"苗栗地方":350,u"臺中地方":400,u"彰化地方":500,u"南投地方":540,u"臺南地方":700,u"高雄地方":800,u"花蓮地方":970,u"雲林地方":630,u"嘉義地方":602,u"臺東地方":950,u"屏東地方":900,u"澎湖地方":880,u"金門地方":890,u"連江地方":209,u"高雄少年及家事":850}
        m = re.search(u'[  \d]*(灣|福建|臺灣)(\S{4,7})法院(民事|家事|暫時|支付)(\S{2})',content)
        return str(court_no[m.group(2)])
    except:
        print "Rex Err:"+m.group(2)
def no_jurytype(content):
    try:
        jurytype_no = {u"婚":11,u"婚再":12,u"婚更":13,u"家聲":21,u"家訴":22,u"家抗":23,u"家陸許":24,u"司家聲":30,u"基家簡":25,u"重家訴":26,u"婚更一":27,u"婚更(一)":28,u"司家調":29,u"家移調":31,u"家簡":32,u"家調裁":33,u"家查":34,u"家聲抗":35,u"家調裁":36,u"家調補":37,u"司家補":38,u"家親聲":39,u"司家簡調":40,u"家簡上":41,u"家小":42,u"家救":43,u"家小上":44,u"家婚聲":45,u"家再":46,u"家他":47,u"訴":48,u"重訴":49,u"家續":50,u"審訴":51,u"聲":52,u"補":90}
        m = re.search(u'(【裁判字號】)(.*?),(.*?),(\d{1,4})[^【】 ]',content)
        return m.group(2)+"_"+str(jurytype_no[m.group(3)])+"_"+m.group(4)
    except:
        print "Rex Err:"+m.group(2)
def judgedate(content):
    m = re.search(u'(【裁判日期】)(\d{6,7})',content)
    return m.group(2)

def getPK(content):
    return no_court(content)+"_"+no_jurytype(content)+"_"+judgedate(content)

cutlist = " <>/:：;；,、＂’，.。！？｢\"\'\\\n\r《》“”!@#$%^&*()".decode("utf-8")  ##列出標點符號，並轉換成utf-8的格式
def cutSentence(content, stopwords): ##放入原始文章路徑, 增加斷詞的list
    sentence = ""
    textList = []
       
    for line in content:
        line = line.strip() ##清除空白
        #line = line.replace(' ', '')
        for word in line:
            if word not in cutlist: #如果文字不是標點符號，就把字加到句子中
                sentence += word
                #print sentence
            else:
                #stopwords =[u'裁判']
                for stopword in stopwords:  #清除關鍵字
                    #print stopword
                    sentence = sentence.replace(stopword,'')
                    #print type(stopword)
                #print sentence
                textList.append(sentence) #把文章句子依標點符號與空白隔開
                sentence = ""
            #for ele in textList: #顯示文章所斷句子
                #print ele
    return textList
    
def ngram(textLists,n): 
    #首參數放處理好的文章(LIST檔，utf-8編碼)次參數放字詞的長度單位，第三個參數放多少頻次以上
    words=[]     #存放擷取出來的字詞
    words_freq={}#存放字詞:計算個數 
    result= []
    for textList in textLists:
        for w in range(len(textList)-(n-1)): #要讀取的長度隨字詞長度改變
            words.append(textList[w:w+n])    #抓取長度w-(n-1)的字串

    for word in words:
        if word not in words_freq:               #如果這個字詞還沒有被放在字典檔中
            words_freq[word] = words.count(word) #就開一個新的字詞，裡面放入字詞計算的頻次
 
    #words_freq = sorted(words_freq.iteritems(),key=operator.itemgetter(1),reverse=True) #change words_freq from dict to list 
    
    #for word in words_freq:
        #if word[1] >= minFreq:
            #result.append(word)
    return words_freq ##回傳一個陣列[(詞,頻次),(詞,頻次)]

def longTermPriority(content, maxTermLength,minTermLength, minFreq):   #目前設定只傳回長詞longTerms
    stopwords=[]
    words_freq = {}
    brokenwords = {}
    mincut = minTermLength-1
    with codecs.open('./ngram/stopwords.txt','r' ,'utf-8') as f :
        readstopwords = f.readlines() #從txt檔讀取每行的停止詞
    for ele in readstopwords:
        stopwords.append(ele.strip())#從每個停止詞中，去除該死的換行符號
    longTermsFreq=[]      #長詞+次數分配
    for i in range(maxTermLength,3,-1): ##字詞數由大至小
        text_list = cutSentence(content,stopwords)  
        words_freq.update(ngram(text_list,i))
        #for word_freq in words_freq:
            #longTermsFreq.append(word_freq) #將長詞和次數加入另外一個list  分成兩個檔儲存的用意是減少迴圈次數
    for key in words_freq:
        if words_freq[key] >= minFreq:
            brokenwords.update({key:words_freq[key]})
    return brokenwords

import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient('localhost',27017)
db = client.test

for dic in db.plaintiff.find():
    #print _uniout.unescape(str(dic),'utf-8') 
    content = dic['plaintiff'] #append all the jurybook to the content
    #print len(content)
    pk = dic['_id']
    #try:
    longTermFreq = longTermPriority(content,8,4,1) ##(input_content, maxTermLength, minTermLength, minFrequence)
    longTermFreq.update({'_id':pk})
    #except:
        #f= codecs.open('./ngram/ngramerr.txt','w+' ,'utf-8')
        #f.write(pk)
        #pass
    try:
        db2 = client.test2
        plaintiff_four = db2['plaintiff_four']
        addmongo = db2.plaintiff_four.insert(longTermFreq)
    except:
        f= codecs.open('./ngram/inserterr.txt','w+' ,'utf-8')
        f.write(pk)
        pass

# Merge dict and Aggregate values for keys

In [5]:
class MyDict(dict):
    def __add__(self, oth):
        #r = self.copy()
        r =self
        try:
            for key, val in oth.items():
                if key in r:
                    r[key] += val # You can custom it here
                else:
                    r[key] = val
        except AttributeError: # In case oth isn't a dict
            return NotImplemented # The convention when a case isn't handled

        return r

import re,codecs
import pymongo
from pymongo import MongoClient
import _uniout
#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
db = client.test2
plaintiff_four = db['plaintiff_four']
#make the collection in the db of test
merge_320_four = db['merge_320_four']
#print the case number in db.dday
dicc ={}
mergedic =MyDict(dicc)
num =1

for dic in db.plaintiff_four.find({'_id':{'$regex':'^320_'}},no_cursor_timeout=True):
    iid = dic['_id']
    del dic['_id']
    mergedic += MyDict(dic)
    mergedic.update({'_id':num})
    aggregate = db.merge_320_four.insert(mergedic)
    f = codecs.open('./log/merge_320_four_log.txt','a+','utf-8')
    log = iid +'\t'+ str(len(mergedic)).decode('utf8')+'\n'
    f.write(log)
    #print iid,len(mergedic)
    num += 1
    #print _uniout.unescape(str(mergedic),'utf-8') 
#print len(mergedic)
f.close()

WriteError: new file allocation failure

In [6]:
f.close()

# Mongo input by location and period of time

In [15]:
import pymongo
from pymongo import MongoClient
client = pymongo.MongoClient('localhost',27017)
db = client.test
#based on location
#for dic in db.samples.find({'_id':{'$regex':'^100_'}}):
    #print dic['_id']
#based on the year
for dic in db.samples.find({'_id':{'$regex':'89(\d{4})$'}}):
    print dic['_id']
    #print _uniout.unescape(str(dic),'utf-8') 

100_88_11_226_890623
100_88_11_234_890124


# Mongo output the sorted fre to txt

In [7]:
#-*- utf-8 -*-
import codecs
import operator
import os
import pymongo
from pymongo import MongoClient
client = pymongo.MongoClient('localhost',27017)
db = client.test2
for ele in db.merge_320_four.find({'_id':2902}):
    words_freq = ele
result= []
text = ''
words_freq = sorted(words_freq.iteritems(),key=operator.itemgetter(1),reverse=True)
for word in words_freq:
    if word[1] >= 2:
        result.append(word)
for i in result:
    text += i[0]+'\t'+str(i[1]).decode('utf8')+'\n'
f = codecs.open('./log/sinica_four_320.txt','w','utf-8')
f.write(text)
f.close()
